In [9]:
%matplotlib notebook
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

In [20]:
#准备数据
df = pd.read_csv('./data/boston.csv', header=0)
print(df.describe())
df = np.array(df)
#df = df.values
for i in range(12):
    df[:,i] = (df[:,i]-df[:,i].min())/(df[:,i].max()-df[:,i].min())
x_data = df[:,:12]
y_data = df[:,12]

             CRIM         ZN       INDUS         CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677082   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

              AGE         DIS         RAD         TAX     PTRATIO       LSTAT  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    68.574901    3.795043    9.549407  408.237154   18.455534   12.653063   
std     28.148861    2.1057

In [32]:
#shape中的None标示行的数量未知
x = tf.placeholder(tf.float32, [None,12], name='X')
y = tf.placeholder(tf.float32,[None,1], name='Y')


In [61]:
#定义模型函数
with tf.name_scope("ModelDefine"):#定义一个命名空间
    w = tf.Variable(tf.random_normal([12,1], stddev=0.01),name='W')
    b = tf.Variable(1.0, name='b')
    def model(x, w, b):
        return tf.matmul(x,w) + b
        #return tf.matmul(w,x) + b
    pred = model(x, w, b)
#模型训练参数
train_epochs = 50
learning_rate = 0.01 #学习率
#定义损失函数
with tf.name_scope("LossFunction"):
    loss_function = tf.reduce_mean(tf.pow(y-pred,2))
    
#tensor board
logdir = './log'
sum_loss_op = tf.summary.scalar('loss', loss_function)
merged = tf.summary.merge_all()

#创建优化器
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)
#模型训练
writer = tf.summary.FileWriter(logdir, sess.graph)# tensor board 
with tf.Session() as sess:
    
    init = tf.global_variables_initializer()
    sess.run(init)
    loss_list = []
    for epoch in range(train_epochs):
        loss_sum = 0.0
        for xs, ys in zip(x_data,y_data):
            xs = xs.reshape(1,12)
            ys = ys.reshape(1,1)
            _, summary_str, loss = sess.run([optimizer, sum_loss_op, loss_function], feed_dict={x:xs, y:ys})
            #writer.add_summary(summary_str, epoch)# tensor board 
            loss_sum = loss_sum + loss

        x_data, y_data = shuffle(x_data, y_data)
        b0temp = b.eval(session=sess)
        w0temp = w.eval(session=sess)
        loss_average = loss_sum/len(y_data)
        print('epoch=', epoch+1, 'loss=', loss_average, 'b=', b0temp, 'w=', w0temp)
        loss_list.append(loss_average)
    plt.plot(loss_list)
#模型使用
n = np.random.randint(506)
print(n)
x_test = x_data[n]

x_test = x_test.reshape(1,12)
with tf.Session() as sess:
    predict = sess.run(pred, feed_dict={x: x_test, w:w0temp, b: b0temp})
    print('predict value:', predict)
    target = y_data[n]
    print('target value:', target)

epoch= 1 loss= 76.33215794203969 b= 15.5298 w= [[-1.3794268 ]
 [ 5.5523963 ]
 [-1.8613904 ]
 [ 3.552536  ]
 [-0.72802347]
 [13.572651  ]
 [ 3.1654232 ]
 [ 5.0118804 ]
 [-2.5589263 ]
 [-3.0905697 ]
 [ 0.30014807]
 [-5.4820995 ]]
epoch= 2 loss= 40.12029134068728 b= 16.428785 w= [[-1.8911006 ]
 [ 5.085448  ]
 [-2.1926048 ]
 [ 4.4554787 ]
 [-0.80891186]
 [17.02099   ]
 [ 2.8029659 ]
 [ 2.8956225 ]
 [-1.1763487 ]
 [-3.303212  ]
 [-2.6638138 ]
 [-9.408634  ]]
epoch= 3 loss= 33.18155845632158 b= 18.787434 w= [[ -2.3402398 ]
 [  4.7717643 ]
 [ -1.510072  ]
 [  4.5286794 ]
 [ -0.49499354]
 [ 20.188305  ]
 [  3.5823288 ]
 [  1.4757262 ]
 [  0.59768677]
 [ -2.731199  ]
 [ -3.3330994 ]
 [-11.656622  ]]
epoch= 4 loss= 30.233858523746193 b= 18.607252 w= [[ -2.8927376 ]
 [  4.0766068 ]
 [ -1.603073  ]
 [  4.0610456 ]
 [ -1.0874047 ]
 [ 21.144575  ]
 [  2.737335  ]
 [ -0.29093492]
 [  0.8346413 ]
 [ -3.3521645 ]
 [ -5.064234  ]
 [-13.760741  ]]
epoch= 5 loss= 28.53059202339802 b= 18.950663 w= [[ -3.27

epoch= 36 loss= 24.06361060399169 b= 27.82029 w= [[ -8.782048  ]
 [  4.2376237 ]
 [  0.24276567]
 [  2.563523  ]
 [ -8.101003  ]
 [ 21.304504  ]
 [  0.1863166 ]
 [-13.81404   ]
 [  5.353356  ]
 [ -6.3859944 ]
 [ -8.491585  ]
 [-19.417267  ]]
epoch= 37 loss= 24.108780874240786 b= 28.535925 w= [[ -8.854175 ]
 [  4.1698265]
 [  0.7050833]
 [  2.8794434]
 [ -7.721232 ]
 [ 21.491795 ]
 [  0.7611401]
 [-13.934131 ]
 [  5.9462423]
 [ -5.8677073]
 [ -8.046857 ]
 [-19.217197 ]]
epoch= 38 loss= 23.9974376379477 b= 28.390182 w= [[ -8.94164   ]
 [  4.018337  ]
 [  0.63002867]
 [  2.6246085 ]
 [ -7.946334  ]
 [ 21.092867  ]
 [  0.67241657]
 [-14.138262  ]
 [  6.024379  ]
 [ -5.9176393 ]
 [ -8.046935  ]
 [-19.103586  ]]
epoch= 39 loss= 23.601122694214705 b= 28.51119 w= [[ -9.038411  ]
 [  4.220444  ]
 [  0.375593  ]
 [  2.790175  ]
 [ -8.207058  ]
 [ 21.083227  ]
 [  0.48638892]
 [-14.057484  ]
 [  5.7934895 ]
 [ -6.294903  ]
 [ -8.179099  ]
 [-19.26675   ]]
epoch= 40 loss= 24.019925588084128 b= 28.

In [42]:
x_test.shape

(1, 12)